In [1]:
import torch
from torchvision import models
from pickle import dump

model = models.vit_l_16(weights='DEFAULT').cuda()
#optimizer = torch.optim.Adam(model.parameters())

In [2]:
IMAGE_SIZE = 224

optimizer_dict = {p: torch.optim.Adam([p], foreach=False) for p in model.parameters()}

def optimizer_hook(parameter) -> None:
  optimizer_dict[parameter].step()
  optimizer_dict[parameter].zero_grad()

for p in model.parameters():
  p.register_post_accumulate_grad_hook(optimizer_hook)

def train(model):
  fake_image = torch.rand(1, 3, IMAGE_SIZE, IMAGE_SIZE).cuda()

  loss = model.forward(fake_image)
  loss.sum().backward()


In [3]:
#del optimizer

# tell CUDA to start recording memory allocations
torch.cuda.memory._record_memory_history(enabled='all')

# train 3 steps
for _ in range(3):
  train(model)

# save a snapshot of the memory allocations
s = torch.cuda.memory._snapshot()
with open(f"snapshot.pickle", "wb") as f:
    dump(s, f)

# tell CUDA to stop recording memory allocations now
torch.cuda.memory._record_memory_history(enabled=None)